In [6]:
import datetime
import json
import pathlib
import re

import numpy as np
import rioxarray

In [7]:
fn = "resize"
size = "50%"
url = "https://eoepca.org/media_portal/images/logo6_med.original.png"

xcengine_config = dict(
  workflow_id = "xceconvert-6",
  container_image_tag = "quay.io/bcdev/xcetest-convert:6",
  environment_file = "environment.yml"
)

In [8]:
ds = rioxarray.open_rasterio(url)

/home/pont/mambaforge/envs/xcube2505a/lib/python3.13/site-packages/rioxarray/_io.py:1143: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  warnings.warn(str(rio_warning.message), type(rio_warning.message))  # type: ignore


In [9]:
ds_small = ds.coarsen(x=2, y=2).mean().astype(np.uint8)

In [10]:
output_basename = re.sub("[.]png$", "", pathlib.Path(url).parts[-1]) + "-resize"
output_pngname = output_basename + ".png"
output_pngpath = pathlib.Path(output_pngname)
output_jsonname = output_basename + ".json"

In [11]:
ds_small.rio.to_raster(output_pngname)

In [12]:
now = datetime.datetime.now(datetime.UTC)
now_iso8601 = re.sub(r"\+00:00$", "Z", now.isoformat(timespec="milliseconds"))

In [13]:
item_stac = {
  "stac_version": "1.1.0",
  "id": f"{output_basename}-{now.timestamp():.9f}",
  "type": "Feature",
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [0, -90],
        [0, 90],
        [360, 90],
        [360, -90],
        [0, -90]
      ]
    ]
  },
  "properties": {
    "start_datetime": "2000-01-01T00:00:00Z",
    "end_datetime": "2001-01-01T00:00:00Z",
    "datetime":	None
  },
  "bbox": [0, -90, 360, 90],
  "assets": {
    output_basename: {
      "type": "image/png",
      "roles": ["data"],
      "href": output_pngname,
      "file:size": output_pngpath.stat().st_size
    }
  },
  "links": [{
    "type": "application/json",
    "rel": "parent",
    "href": "catalog.json"
  }, {
    "type": "application/json",
    "rel": "root",
    "href": "catalog.json"
  }]
}

item_stac["properties"] |= {"xcengine_config:" + k: v for k, v in xcengine_config.items()}

with open(output_jsonname, "w") as fh:
    json.dump(item_stac, fh, indent=2)

In [86]:
catalog_stac = {
  "stac_version": "1.1.0",
  "id": "catalog",
  "type": "Catalog",
  "description": "Root catalog",
  "links": [{
    "type": "application/geo+json",
    "rel": "item",
    "href": output_jsonname
  }, {
    "type": "application/json",
    "rel": "root",
    "href": "catalog.json"
  }]
}

with open("catalog.json", "w") as fh:
    json.dump(catalog_stac, fh, indent=2)